In [14]:
# import numpy as np
# import random

In [36]:
# genetic algorithm search for continuous function optimization
from numpy.random import randint
from numpy.random import rand

# objective function
def objective(x):
    return x[0]**2.0 + x[1]**2.0

# decode bitstring to numbers
def decode(bounds, n_bits, bitstring):
    decoded = list()
    largest = 2**n_bits
    for i in range(len(bounds)):
        # extract the substring
        start, end = i * n_bits, (i * n_bits)+n_bits
        substring = bitstring[start:end]
        # convert bitstring to a string of chars
        chars = ''.join([str(s) for s in substring])
        # convert string to integer
        integer = int(chars, 2)
        # scale integer to desired range
        value = bounds[i][0] + (integer/largest) * (bounds[i][1] - bounds[i][0])
        # store
        decoded.append(value)
    return decoded

# tournament selection
def selection(pop, scores, k=3):
    # first random selection
    selection_ix = randint(len(pop))
    for ix in randint(0, len(pop), k-1):
        # check if better (e.g. perform a tournament)
        if scores[ix] < scores[selection_ix]:
            selection_ix = ix
    return pop[selection_ix]

# crossover two parents to create two children
def crossover(p1, p2, r_cross):
    # children are copies of parents by default
    c1, c2 = p1.copy(), p2.copy()
    # check for recombination
    if rand() < r_cross:
        # select crossover point that is not on the end of the string
        pt = randint(1, len(p1)-2)
        # perform crossover
        c1 = p1[:pt] + p2[pt:]
        c2 = p2[:pt] + p1[pt:]
    return [c1, c2]

# mutation operator
def mutation(bitstring, r_mut):
    for i in range(len(bitstring)):
        # check for a mutation
        if rand() < r_mut:
            # flip the bit
            bitstring[i] = 1 - bitstring[i]

# genetic algorithm
def genetic_algorithm(objective, bounds, n_bits, n_iter, n_pop, r_cross, r_mut):
    # initial population of random bitstring
    pop = [randint(0, 2, n_bits*len(bounds)).tolist() for _ in range(n_pop)]
    # keep track of best solution
    best, best_eval = 0, objective(decode(bounds, n_bits, pop[0]))
    # enumerate generations
    for gen in range(n_iter):
        # decode population
        decoded = [decode(bounds, n_bits, p) for p in pop]
        # evaluate all candidates in the population
        scores = [objective(d) for d in decoded]
        # check for new best solution
        for i in range(n_pop):
            if scores[i] < best_eval:
                best, best_eval = pop[i], scores[i]
                print(">%d, new best f(%s) = %f" % (gen,  decoded[i], scores[i]))
        # select parents
        selected = [selection(pop, scores) for _ in range(n_pop)]
        # create the next generation
        children = list()
        for i in range(0, n_pop, 2):
            # get selected parents in pairs
            p1, p2 = selected[i], selected[i+1]
            # crossover and mutation
            for c in crossover(p1, p2, r_cross):
                # mutation
                mutation(c, r_mut)
                # store for next generation
                children.append(c)
        # replace population
        pop = children
    return [best, best_eval]

# define range for input
bounds = [[-5.0, 5.0], [-5.0, 5.0]]
# define the total iterations
n_iter = 100
# bits per variable
n_bits = 16
# define the population size
n_pop = 100
# crossover rate
r_cross = 0.9
# mutation rate
r_mut = 1.0 / (float(n_bits) * len(bounds))
# perform the genetic algorithm search
best, score = genetic_algorithm(objective, bounds, n_bits, n_iter, n_pop, r_cross, r_mut)
print('Done!')
decoded = decode(bounds, n_bits, best)
print('f(%s) = %f' % (decoded, score))

>0, new best f([0.799560546875, 2.042236328125]) = 4.810026
>0, new best f([0.001373291015625, 1.35284423828125]) = 1.830189
>0, new best f([0.28839111328125, 0.25238037109375]) = 0.146865
>1, new best f([0.15838623046875, 0.011444091796875]) = 0.025217
>2, new best f([0.0091552734375, 0.011444091796875]) = 0.000215
>12, new best f([-0.0006103515625, 0.005035400390625]) = 0.000026
>15, new best f([-0.00335693359375, 0.00152587890625]) = 0.000014
>26, new best f([-0.003204345703125, 0.0006103515625]) = 0.000011
>26, new best f([-0.00030517578125, 0.00213623046875]) = 0.000005
>28, new best f([-0.00030517578125, 0.0018310546875]) = 0.000003
>28, new best f([-0.00030517578125, 0.0006103515625]) = 0.000000
>30, new best f([-0.00030517578125, 0.0]) = 0.000000
>38, new best f([-0.000152587890625, 0.000152587890625]) = 0.000000
>38, new best f([-0.000152587890625, 0.0]) = 0.000000
Done!
f([-0.000152587890625, 0.0]) = 0.000000


In [27]:
# # File: genetic.py
# #    from chapter 3 of _Genetic Algorithms with Python_
# #
# # Author: Clinton Sheppard <fluentcoder@gmail.com>
# # Copyright (c) 2016 Clinton Sheppard
# #
# # Licensed under the Apache License, Version 2.0 (the "License").
# # You may not use this file except in compliance with the License.
# # You may obtain a copy of the License at
# #   http://www.apache.org/licenses/LICENSE-2.0
# #
# # Unless required by applicable law or agreed to in writing, software
# # distributed under the License is distributed on an "AS IS" BASIS,
# # WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or
# # implied.  See the License for the specific language governing
# # permissions and limitations under the License.

# import random
# import statistics
# import sys
# import time


# def _generate_parent(length, geneSet, get_fitness):
#     genes = []
#     while len(genes) < length:
#         sampleSize = min(length - len(genes), len(geneSet))
#         genes.extend(random.sample(geneSet, sampleSize))
#     fitness = get_fitness(genes)
#     return Chromosome(genes, fitness)


# def _mutate(parent, geneSet, get_fitness):
#     childGenes = parent.Genes[:]
#     index = random.randrange(0, len(parent.Genes))
#     newGene, alternate = random.sample(geneSet, 2)
#     childGenes[index] = alternate if newGene == childGenes[index] else newGene
#     fitness = get_fitness(childGenes)
#     return Chromosome(childGenes, fitness)


# def get_best(get_fitness, targetLen, optimalFitness, geneSet, display):
#     random.seed()

#     def fnMutate(parent):
#         return _mutate(parent, geneSet, get_fitness)

#     def fnGenerateParent():
#         return _generate_parent(targetLen, geneSet, get_fitness)

#     for improvement in _get_improvement(fnMutate, fnGenerateParent):
#         display(improvement)
#         if not optimalFitness > improvement.Fitness:
#             return improvement


# def _get_improvement(new_child, generate_parent):
#     bestParent = generate_parent()
#     yield bestParent
#     while True:
#         child = new_child(bestParent)
#         if bestParent.Fitness > child.Fitness:
#             continue
#         if not child.Fitness > bestParent.Fitness:
#             bestParent = child
#             continue
#         yield child
#         bestParent = child


# class Chromosome:
#     def __init__(self, genes, fitness):
#         self.Genes = genes
#         self.Fitness = fitness


# class Benchmark:
#     @staticmethod
#     def run(function):
#         timings = []
#         stdout = sys.stdout
#         for i in range(100):
#             sys.stdout = None
#             startTime = time.time()
#             function()
#             seconds = time.time() - startTime
#             sys.stdout = stdout
#             timings.append(seconds)
#             mean = statistics.mean(timings)
#             if i < 10 or i % 10 == 9:
#                 print("{} {:3.2f} {:3.2f}".format(
#                     1 + i, mean,
#                     statistics.stdev(timings, mean) if i > 1 else 0))

In [34]:
# import random

# # constants
# POP_SIZE = 6
# GENOME_SIZE = 5
# GENERATIONS = 10
# PROB_MUTATION = 0.1
# TOURAMENT_SIZE = 3
# pop = []
# # functions
# def population_initialise():
#     return([(''.join(random.choice(range(10, 1000)) for i in range(GENOME_SIZE))) 
#                         for i in range(POP_SIZE)])

# def fitness(chromosome):
#     return str(chromosome).count("1")

# def selection(population):
#     tournament = [random.choice(population) for i in range(TOURAMENT_SIZE)]
#     fitnesses = [fitness(tournament[i] for i in range(TOURAMENT_SIZE))]
#     return tournament[fitnesses.index(max(fitnesses))]

# def crossing_over(parent1, parent2):
#     parent1, parent2 = str(parent1), str(parent2)
#     xo_point = random.randint(1, GENOME_SIZE-1)
#     return([
#             parent1[0:xo_point] + parent2[xo_point:GENOME_SIZE],
#             parent2[0:xo_point] + parent1[xo_point:GENOME_SIZE] ])

# def mutate_mech(bit):
#     bit = bit
#     if bit == 0:
#         return 0
#     else:
#         return bit

# def mutation(chromosome):
#     chromosome = chromosome
#     for i in range(GENOME_SIZE):
#         if random.random() < PROB_MUTATION:
#             chromosome = chromosome[:i]  + chromosome[i+1:]
#     return(chromosome)

# def print_population(population):            
#     fitnesses = [fitness(population[i]) for i in range(POP_SIZE)]
#     print(list(zip(population, fitnesses)))
    
# # Algorithm
# random.seed()
# population = population_initialise()

# for gen in range(GENERATIONS):
#     print("Genneration " + str(gen))
#     print_population(population)
#     if max([fitness(population[i]) for i in range(POP_SIZE)]) == GENOME_SIZE:
#         break;
#     nextgen_population = []
#     for i in range(int(POP_SIZE/2)):
#         parent1 = selection(population)
#         parent2 = selection(population)
#         offspring = crossing_over(parent1, parent2)
#         nextgen_population.append(mutation(offspring[0]))
#         nextgen_population.append(mutation(offspring[1]))
#     population = nextgen_population
    

TypeError: sequence item 0: expected str instance, int found